In [1]:
pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 804.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import gradio as gr

# Load synthetic data
from google.colab import files


# Upload the JSON file
uploaded = files.upload()

# Load the JSON file into a dictionary
json_data = json.load(open(list(uploaded.keys())[0]))

# Convert the dictionary to a DataFrame
df = pd.DataFrame(json_data)

# Function to calculate debt-to-income ratio
def calculate_dti(expenditure, income):
    return round(expenditure / income, 2)

# Function to provide suggestions for credit improvement
def credit_improvement_suggestions(credit_score, dti):
    suggestions = []

    # Check credit score and give suggestions based on range
    if credit_score < 580:
        suggestions.append("Your credit score is poor. Focus on paying bills on time.")
        suggestions.append("Reduce credit card balances and avoid new credit inquiries.")
    elif 580 <= credit_score < 670:
        suggestions.append("Your credit score is fair. Work on lowering your credit utilization below 30%.")
        suggestions.append("Consider setting up automatic payments to avoid late payments.")
    elif 670 <= credit_score < 740:
        suggestions.append("Your credit score is good. Keep up with timely payments and aim for even lower credit utilization.")
    else:
        suggestions.append("Your credit score is excellent. Keep up the good work! Continue monitoring your credit and maintaining low balances.")

    # Check DTI ratio and give suggestions based on range
    if dti > 0.4:
        suggestions.append("Your debt-to-income ratio is high. Consider paying down high-interest debt to lower it.")
    else:
        suggestions.append("Your debt-to-income ratio is healthy. Focus on maintaining low balances and building savings.")

    return suggestions

# Function to find matching cases and give tailored suggestions
def predict_credit_improvement(credit_score, expenditure, savings, income):
    dti = calculate_dti(expenditure, income)

    # Generate suggestions for credit improvement
    suggestions = credit_improvement_suggestions(credit_score, dti)

    # Generate comparative stats
    comparison = {
        'Your Credit Score': credit_score,
        'Your DTI Ratio': dti,
        'Average DTI Ratio in Data': df['debt_to_income_ratio'].mean(),
        'Your Savings': savings,
        'Average Savings in Data': df['savings'].mean(),
    }

    # Generate a plot comparing DTI ratios
    fig, ax = plt.subplots()
    ax.bar(['Your DTI', 'Average DTI'], [dti, df['debt_to_income_ratio'].mean()])
    ax.set_ylabel('Debt-to-Income Ratio')
    plt.tight_layout()

    # Save plot to a file and display it in Gradio
    plot_filename = "dti_plot.png"
    plt.savefig(plot_filename)
    plt.close()

    return suggestions, comparison, plot_filename

# Gradio interface
def gradio_interface(credit_score, expenditure, savings, income):
    suggestions, comparison, plot_filename = predict_credit_improvement(credit_score, expenditure, savings, income)

    # Prepare the suggestions output
    suggestion_text = "\n".join(f"- {suggestion}" for suggestion in suggestions)

    # Prepare comparison text
    comparison_text = "\n".join(f"{key}: {value}" for key, value in comparison.items())

    return suggestion_text, comparison_text, plot_filename

# Define Gradio inputs and outputs
inputs = [
    gr.Slider(minimum=300, maximum=850, step=1, label="Credit Score"),
    gr.Number(label="Monthly Expenditure"),
    gr.Number(label="Savings"),
    gr.Number(label="Monthly Income")
]

outputs = [
    gr.Textbox(label="Suggestions for Improving Credit Score"),
    gr.Textbox(label="Comparative Analysis"),
    gr.Image(label="Debt-to-Income Ratio Plot")
]

# Create the Gradio interface
interface = gr.Interface(fn=gradio_interface, inputs=inputs, outputs=outputs, title="Credit Score Improvement Tool")

# Launch the Gradio app
interface.launch()


Saving modified_synthetic_personal_finance_data.json to modified_synthetic_personal_finance_data.json
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e6db8f413ec0d3034d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
